RESTRICTIONS
===
Restrictions $\text{res}^G_H \colon A(G) \to A(H)$ in the Burnside Tambara functor are given by 
$$
    \text{res}^G_H([X]) = [i_H^*(X)],
$$
where $i_H^* \colon \text{Set}^G \to \text{Set}^H$ is the forgetful functor. This is simple to describe in the Ghost ring. For an arbitrary class function $f \colon \text{Sub}(G)/G \to \mathbb{Z}$, 
$$
    \text{res}^G_H(f)([K]) = f([K]).
$$


In [ ]:
# A function to compute there striction from G to H in the ghost ring
#
# INPUT 
# H,G are GAP groups such that H is a subgroup of G
#
# OUTPUT
# A matrix M such that right-multiplication by M implements the restriction from G to H
# in the ghost ring. 
#
# DAVID M: I made the order of the inputs opposite the transfer and norm here. Do we want that?
def restriction_marks(G,H):
    # if H is not a subgroup of G, then throw an error 
    if not gap.IsSubgroup(G,H): 
        raise ValueError("You fool! H must be a subgroup of G") # maybe the error could be nicer

    tomG = G.TableOfMarks() # create the table of marks of G
    matG = matrix(ZZ, tomG.MatTom()) # table of marks as a matrix 
    nG = matG.dimensions()[0] # dimensions of table of marks
    repsG = [tomG.RepresentativeTom(n+1) for n in range(nG)] # reps of conjugacy classes

    tomH = H.TableOfMarks() # create the table of marks of H
    matH = matrix(ZZ, tomH.MatTom()) # table of marks as a matrix
    nH = matH.dimensions()[0]
    repsH = [tomH.RepresentativeTom(n+1) for n in range(nH)] # reps of conjugacy classes

    return matrix(ZZ,nG,nH,lambda i,j : G.IsConjugate(repsG[i],repsH[j]).sage())


# A function to compute there striction from G to H in the ghost ring
#
# INPUT 
# H,G are GAP groups such that H is a subgroup of G
#
# OUTPUT
# A matrix M that represents the restriction from A(G) to A(H).
# Right-multiplication by this matrix M of a row vector in A(G) represents restriction to A(H).
# These Burnside rings are given the additive bases of orbits (as row vectors), 
# presented in the same order they appear in the rows of the GAP table of marks.
#
# DAVID M: I made the order of the inputs opposite the transfer and norm here. Do we want that?    
def restriction(G,H):
    # if H is not a subgroup of G, then throw an error 
    if not gap.IsSubgroup(G,H): 
        raise ValueError("You fool! H must be a subgroup of G") # maybe the error could be nicer

    tomG = G.TableOfMarks() # create the table of marks of G
    matG = matrix(ZZ, tomG.MatTom()) # table of marks as a matrix 
    
    tomH = H.TableOfMarks() # create the table of marks of H
    matH = matrix(ZZ, tomH.MatTom()) # table of marks as a matrix

    return matG*restriction_marks(G,H)*matH^(-1)
    

In [117]:
Q = gap("QuaternionGroup(8)")
subgroupsQ = [H.Representative() for H in Q.ConjugacyClassesSubgroups()]
e = subgroupsQ[0]
C2 = subgroupsQ[1]
I = subgroupsQ[2]
J = subgroupsQ[3]
K = subgroupsQ[4]

restriction(Q,I)


[2 0 0]
[0 2 0]
[0 0 2]
[0 1 0]
[0 1 0]
[0 0 1]

TRANSFERS
===
Transfers $\text{tr}_H^G \colon A(H) \to A(G)$ in the Burnside Tambara functor are given by 
$$ 
    \text{tr}_H^G([X]) = \left[X \times_H G\right].
$$
Transfers also have a nice formula in the ghost ring. For an arbitrary class function $f \colon \text{Sub(G)}/G \to \mathbb{Z}$, 
$$
    \text{tr}_H^G(f)([K]) = \sum_{\substack{gH \in G/H \\ K^g \subseteq H}} f([K^g]).
$$

In [ ]:
# A function to compute the transfer from H to G in the Burnside ring
#
# INPUT
# H,G are GAP groups such that H is a subgroup of G
#
# OUTPUT
# A matrix M that represents the transfer from A(H) to A(G).
# Right-multiplication by this matrix M of a row vector in A(H) represents transfer to A(G).
# These Burnside rings are given the additive bases of orbits (as row vectors), 
# presented in the same order they appear in the rows of the GAP table of marks.
#
def transfer(H,G):
    # if H is not a subgroup of G, then throw an error 
    if not gap.IsSubgroup(G,H): 
        raise ValueError("You fool! H must be a subgroup of G") # maybe the error could be nicer
    
    tomG = G.TableOfMarks() # create the table of marks of G
    matG = matrix(ZZ, tomG.MatTom()) # table of marks as a matrix 
    nG = matG.dimensions()[0] # dimensions of table of marks
    repsG = [tomG.RepresentativeTom(n+1) for n in range(nG)] # reps of conjugacy classes

    tomH = H.TableOfMarks() # create the table of marks of H
    matH = matrix(ZZ, tomH.MatTom()) # table of marks as a matrix
    nH = matH.dimensions()[0]
    repsH = [tomH.RepresentativeTom(n+1) for n in range(nH)] # reps of conjugacy classes

    return matrix(ZZ,nH,nG,lambda i,j: G.IsConjugate(repsH[i],repsG[j]).sage()) 
        # create a matrix where the i,j entry is 1 if G \times_H H/K_i = G/L_j
        # G.IsConjugate(K1,K2) returns a boolean GAP object; .sage() converts it to normal True/False
        # then python interprets True/False as 0 or 1 in a matrix.


# A function that implements the transfer on the ghost ring (as above)
#   This is strictly speaking not necessary, because you could just get 
#   this matrix by basis-changing the matrix of transfer(H,G) using the tables of marks.
#   In fact, it's almost certainly more efficient to get it that way
#
# INPUT 
# H,G are GAP groups such that H is a subgroup of G
#
# OUTPUT
# A matrix M such that right-multiplication by M implements the transfer from H to G
# in the ghost ring. 
#
def transfer_marks(H,G):
    # if H is not a subgroup of G, then throw an error 
    if not gap.IsSubgroup(G,H): 
        raise ValueError("You fool! H must be a subgroup of G") # maybe the error could be nicer

    tomG = G.TableOfMarks() # create the table of marks of G
    matG = matrix(ZZ, tomG.MatTom()) # table of marks as a matrix 
    nG = matG.dimensions()[0] # dimensions of table of marks
    repsG = [tomG.RepresentativeTom(n+1) for n in range(nG)] # reps of conjugacy classes

    tomH = H.TableOfMarks() # create the table of marks of H
    matH = matrix(ZZ, tomH.MatTom()) # table of marks as a matrix
    nH = matH.dimensions()[0]
    repsH = [tomH.RepresentativeTom(n+1) for n in range(nH)] # reps of conjugacy classes

    cosetReps = [C.Representative() for C in G.LeftCosets(H)] # coset representatives for G/H

    tr = matrix(ZZ,nH,nG) # make a matrix of all zeros of the appropriate dimensions

    for i in range(nH):
        L = repsH[i]
        for j in range(nG): 
            K = repsG[j]
            tr[i,j] = sum(1 for g in cosetReps if H.IsConjugate(L,K.ConjugateGroup(g)))
    return tr

In [ ]:
S5 = gap("SymmetricGroup(5)")
S4 = gap("SymmetricGroup(4)")
S3 = gap("SymmetricGroup(3)")
C2 = gap("SymmetricGroup(2)")
e = gap("SymmetricGroup(1)")

matS3 = matrix(ZZ,S3.TableOfMarks().MatTom())
matC2 = matrix(ZZ,C2.TableOfMarks().MatTom())
assert matC2*transfer_marks(C2,S3)*matS3^(-1) == transfer(C2,S3)

matS4 = matrix(ZZ,S4.TableOfMarks().MatTom())
matS3*transfer_marks(S3,S4)*matS4^(-1) == transfer(S3,S4)

matS5 = matrix(ZZ,S5.TableOfMarks().MatTom())
assert matS4*transfer_marks(S4,S5)*matS5^(-1) == transfer(S4,S5) 

assert vector([1])*transfer(e,S5) == vector(ZZ,19,{0:1})
assert transfer(e,S3)*transfer(S3,S5) == transfer(e,S5)

NORMS
===

The norm $\text{nm}_H^G \colon A(H) \to A(G)$ the Burnside Tambara functor is given by 
$$
    \text{nm}_H^G(X) = \text{Set}^H(G,X),
$$
where the right-hand side is the $H$-equivariant functions from $G$ to $X$. This becomes a $G$-set via $(g \cdot f)(x) = f(xg).$

We can also encode these norms in the table of marks. For an arbitrary class function $f \colon \text{Sub}(G)/G \to \mathbb{Z}$, 
$$
    \text{nm}_H^G(f)([K]) = \prod_{KgH \in K \backslash G / H} f([K^g \cap H]).
$$

In [51]:
# A function to compute the norm from H to G in the Burnside ring
#
# INPUT
# H, G are GAP groups
# f is a vector defining a linear combination of H-orbits
#   The order of orbits is the same as the rows of the table of marks of H, as given by GAP
#   For example, if H is the quaternion group Q_8, then the vector f = (0,1,-1,0,2,0)
#   represents the element Q_8/C_2 - Q_8/I + 2 Q_8/K, where I = <i> and K = <k>. 
#   
#   For Q_8, the GAP table of marks presents conjugacy classes of subgroups in the order 
#   e, C_2, I, J, K, Q_8, where I = <i>, J = <j>, and K = <k>. 
#
# OUTPUT 
# a vector n encoding a linear combination of G-orbits, with order of orbits as above
#
# EXAMPLES 
#  tbd
#
def norm(H,G,f):
    # if H is not a subgroup of G, then throw an error 
    if not gap.IsSubgroup(G,H): 
        raise ValueError("You fool! H must be a subgroup of G") # maybe the error could be nicer

    tomG = G.TableOfMarks() # create the table of marks of G
    matG = matrix(ZZ, tomG.MatTom()) # table of marks as a matrix 
    nG = matG.dimensions()[0] # dimension of table of marks
    repsG = [tomG.RepresentativeTom(n+1) for n in range(nG)] # reps of conjugacy classes

    tomH = H.TableOfMarks() # create the table of marks of H
    matH = matrix(ZZ, tomH.MatTom()) # table of marks as a matrix
    nH = matH.dimensions()[0]
    repsH = [tomH.RepresentativeTom(n+1) for n in range(nH)] # reps of conjugacy classes

    marksf = f * matH # convert the vector f of coefficients into an element of the ghost ring
    nm = vector(ZZ, [1]*nG) # empty product is 1; setup for looping
    for i in range(nG):
        K = repsG[i] # we're finding the component indexed by K
        for(g,_) in G.DoubleCosetRepsAndSizes(K,H): # only need the reps of double cosets
            HcapKg = gap.Intersection(H,K.ConjugateGroup(g)) # form the intersection of H and the conjugate of K by g
            index = next(j for j,r in enumerate(repsH) if H.IsConjugate(r, HcapKg)) # find the index of the subgroup of H that represents H \cap K^g
            nm[i] *= marksf[index] # multiply the [K] component of the norm by marksf([H \cap K^g])
    return matG.solve_left(nm) # convert back into a list of coefficients

In [52]:
S5 = gap("SymmetricGroup(5)")
S4 = gap("SymmetricGroup(4)")
C2 = gap("SymmetricGroup(2)")
e = gap("SymmetricGroup(1)")
# norm from e to C2 of 0 is 0 and of 1 is 1
# in general norm from e to C2 of a positive a is (a choose 2, a)
assert norm(e,C2,vector([0])) == vector([0,0])
assert norm(e,C2,vector([1])) == vector([0,1])
assert norm(e,C2,vector([4])) == vector([6,4])

# norm from e to S5 of 0 is 0 and of 1 is 1
# these are slow, comment out if you don't want to wait 4s
assert norm(e,S5,vector([0])) == vector(ZZ,[0] * 19)
assert norm(e,S5,vector([1])) == vector(ZZ, 19, {18:1})

norm(S4, S5, vector(ZZ, 11, {3:1,4:1}))

(4341, 0, 264, 11, 14, 0, 0, 2, 0, 0, 6, 0, 3, 0, 0, 0, 0, 0, 0)